In [8]:
import sys
!{sys.executable} -m pip install playwright

   ---------------------------------------- 0.0/36.6 MB ? eta -:--:--
   --------- ------------------------------ 8.4/36.6 MB 46.7 MB/s eta 0:00:01
   ----------------- ---------------------- 16.0/36.6 MB 40.5 MB/s eta 0:00:01
   -------------------------- ------------- 24.1/36.6 MB 40.4 MB/s eta 0:00:01
   ----------------------------------- ---- 32.5/36.6 MB 40.2 MB/s eta 0:00:01
   ---------------------------------------- 36.6/36.6 MB 37.3 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [pyee]
   -------------------- ------------------- 1/2 [playwright]
   -------------------- ------------------- 1/2 [playwright]
   -------------------- ------------------- 1/2 [playwright]
   -------------------- ------------------- 1/2 [playwright]
   -------------------- ------------------- 1/2 [playwright]
   -------------------- ------------------- 1/2 [playwright]
   -------------------- ------------------- 1/2 [playwright]
   -------------------- ------------------- 1/2 [play

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import pandas as pd

In [4]:
Bcorp = pd.read_csv(r"C:\Users\engele016\OneDrive - Gemeente Amsterdam\Documenten\Github\Impact-data-scraper\B Corp Impact Data.csv")

C:\Users\engele016\AppData\Local\Temp\ipykernel_18896\1910107611.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Bcorp = pd.read_csv(r"C:\Users\engele016\OneDrive - Gemeente Amsterdam\Documenten\Github\Impact-data-scraper\B Corp Impact Data.csv")


In [5]:
Bcorp = Bcorp[["company_name","date_first_certified", "date_certified", "current_status","website", "b_corp_profile", "sector", "city", "country", "other_countries_of_operation" ]]

In [6]:
filtered = Bcorp[
    (Bcorp["city"] == "Amsterdam") |
    (Bcorp["other_countries_of_operation"] == "The Netherlands")|
    (Bcorp["city"] == "AMSTERDAM")]

In [7]:
filtered = filtered.drop_duplicates(subset="company_name", keep="first")

In [9]:
import re
import pandas as pd
from playwright.sync_api import sync_playwright

ADDRESS_REGEX = re.compile(
    r"[A-Z][a-zA-ZÀ-ÿ\s]+ \d+[A-Z]?\s*\n?\s*"
    r"\d{4}\s?[A-Z]{2}\s+[A-Z][a-zA-ZÀ-ÿ\s]+"
)

COOKIE_SELECTORS = [
    # Common text buttons (EN/NL)
    "text=Accept all", "text=Accept All", "text=Accept",
    "text=I agree", "text=Agree",
    "text=Alles accepteren", "text=Accepteren", "text=Akkoord", "text=Toestaan",
    "text=Alle cookies accepteren", "text=Ik ga akkoord",

    # Common CMP-specific selectors (best-effort; harmless if not found)
    "#onetrust-accept-btn-handler",
    "button#onetrust-accept-btn-handler",
    "button[aria-label*='accept' i]",
    "button[class*='accept' i]",
]

def clean_url(url: str) -> str:
    url = url.strip()
    if not url.startswith("http"):
        url = "https://" + url
    return url.rstrip("/")

def _normalize_text(s: str) -> str:
    s = s.replace("\r", "\n")
    s = re.sub(r"\n+", "\n", s)
    return s

def extract_address_from_text(text: str):
    text = _normalize_text(text)
    m = ADDRESS_REGEX.search(text)
    return m.group(0).strip() if m else None

def playwright_get_address(url: str, timeout_ms: int = 20000):
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        context
